<a href="https://colab.research.google.com/github/Jerry086/CS6140CarbonMapping/blob/main/EE_Python_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Earth Engine for Python Developers

This notebook tutorial gives an introduction to analyzing satellite imagery with Earth Engine, using the Earth Engine Python [client library](https://github.com/google/earthengine-api). The analysis is similar to the one found in the [Javascript Earth Engine API tutorial](https://developers.google.com/earth-engine/tutorials/tutorial_api_01). You will see a 🐍 where the Python approach deviates from the Javascript approach.


# Setup 🐍



⚠️ If viewing this content after the 2022 Geo for Good Summit (i.e. October 2022),
refer to the Earth Engine documentation for current guidance on [authentication using Python](https://developers.google.com/earth-engine/guides/python_install#authentication).


## Authentication

Authenticate and authorize access to Earth Engine.

In [ ]:
COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS = None

import ee
import google
import os

if COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS is None:
  print("Authenticating using Notebook auth...")
  if os.path.exists(ee.oauth.get_credentials_path()) is False:
    ee.Authenticate()
  else:
    print('\N{check mark} '
          'Previously created authentication credentials were found.')
  ee.Initialize()
else:
  print('Authenticating using Colab auth...')
  # Authenticate to populate Application Default Credentials in the Colab VM.
  google.colab.auth.authenticate_user()
  # Create credentials needed for accessing Earth Engine.
  credentials, auth_project_id = google.auth.default()
  # Initialize Earth Engine.
  ee.Initialize(credentials, project=COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS)
print('\N{check mark} Successfully initialized!')

## Install & import Python packages



### About the earthengine_jupyter package
This notebook uses the [earthengine-jupyter](https://github.com/google/earthengine-jupyter) Python package, which provides helpful tools for working with Earth Engine from within a [Jupyter](https://jupyter.org/) environment. It currently enables use of interactive map and inspector widgets.

⚠️ Note that the package is **experimental** and is **not ready for production use**.



The following code block installs the `earthengine_jupyter` package, if it doesn't already exist on the server.

In [ ]:
try:
  import ee_jupyter
except ModuleNotFoundError:
  print('ee_jupyter was not found. Installing now...')
  result = os.system('pip -q install earthengine-jupyter')
  import ee_jupyter
print(f'ee_jupyter (version {ee_jupyter.__version__}) '
        f'is installed.')

Import other Python packages used throughout this notebook.

In [ ]:
import altair as alt
from ee_jupyter.colab import set_colab_output_cell_height
from ee_jupyter.ipyleaflet import Map
from ee_jupyter.ipyleaflet import Inspector
from ee_jupyter.layout import MapWithInspector
import ipyleaflet
import ipywidgets as widgets
from IPython.display import HTML
import math
import pandas as pd
from pprint import pprint  # for pretty printing

# Visualizing Images and Image Bands

First we will explore the Shuttle Radar Topography Mission (STRM) dataset. Each pixel in the [STRM dataset](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4) represents the elevation of that point on the Earth, measured in meters above sea level. The STRM dataset is a Digital Elevation Model (DEM).

In [ ]:
# Instantiate an image object for the STRM elevation dataset.
dem = ee.Image('CGIAR/SRTM90_V4')

Print out a text representation of the image object.

In [ ]:
# Sets how big the output is in the cell (since this is a big print statement).
set_colab_output_cell_height(300)
pprint(dem.getInfo())

## Display a map 🐍

Let's create an interactive map that is centered on the venue of Geo for Good and then add the STRM dataset layer to the map.

First define coordinates as a list, and then use that to define the initial parameters of the map.

In [ ]:
# Coordinates of Geo for Good venue in Mountain View.
location_lonlat= [-122.0648754, 37.4225866]
map_init_params = {
    'center': list(reversed(location_lonlat)), # <lat,lon> ordering
    'zoom': 10
}

Note the use of `reversed()` to swap coordinate ordering from the <lon,lat> convention used by [GeoJSON spec](https://www.rfc-editor.org/rfc/rfc7946#section-3.1.1) (which Earth Engine follows) to the <lat,lon> convention used by the underlying [leaflet mapping library](https://leafletjs.com/reference.html#latlng).

Having a variable name on the last line of a code cell, causes the value of the variable to be printed. For example:

In [ ]:
map_init_params

Instantiate an interactive map object and display it.

In [ ]:
map1 = Map(**map_init_params)
map1

Add a layer to the map that shows a greyscale visualization of the DEM/elevation dataset.  1000 meters is a reasonable maximum elevation for Mountain View, but you may want to choose a different value that is appropriate for the region you are viewing.

In [ ]:
map1.addLayer(dem, {'min': 0, 'max': 1000}, 'elevation (greyscale)')

Note that the layer does not complete cover the basemap along the shoreline. This is due to underlying dataset (SRTM) which is masked out in those locations. This is because the elevation is at or below sea level. In Mountain View, these areas are likely to be the wetlands near the bay.

We can style image layers with a custom color palette. The following adds a second layer to the map, which you can toggle on and off using the layer selector control, located at the top right of the map.

In [ ]:
map1.addLayer(
  dem,
  {'min': 0, 'max': 1000, 'palette': ['grey', 'blue']},
  'elevation (custom palette)'
)

In [ ]:
map1

**Extra practice ✅:** The Earth Engine data catalog contains [numerous DEMs](https://developers.google.com/earth-engine/datasets/tags/dem). Experiment with replacing the current DEM (SRTM) with another DEM.

**Extra practice ✅:** You can change `location_lonlat` to center the map on a different region.

# Computations using Images

It is often useful to process images before displaying them. This section shows one method of applying an algorithm (`ee.Terrain.slope`) to an image to create a new (derived) image object (`slope`).

In [ ]:
# Apply an algorithm to an image.
slope = ee.Terrain.slope(dem)

# Display the result.
map2a = Map(**map_init_params)
map2a.addLayer(dem, {'min': 0, 'max': 1000}, 'elevation [meters]')
# Slope is calculated in degrees so 30 degrees was chosen as the max for this
# area. In general, is rare to go above 45 degrees.
map2a.addLayer(slope, {'min': 0, 'max': 30}, 'slope [degrees]')
map2a

## Using a map inspector 🐍

When working with geospatial (i.e. map-referenced) data, it is often useful to have an *inspector* tool for querying information at a particular point. Most GIS software provides this type of functionality. We will use the `Inspector` object, which is defined in the [google/earthengine-jupyter](https://github.com/google/earthengine-jupyter/blob/main/ee_jupyter/ipyleaflet.py#L87) library.

In [ ]:
inspector2a = Inspector(map2a)
widgets.HBox([map2a, inspector2a])

Creating a map with an inspector panel is common, so we created a helper class to instantiate it.

In [ ]:
map_panel_2 = MapWithInspector(**map_init_params)
map_panel_2

Layers can be added to the map attribute of the inspector panel.

In [ ]:
map_panel_2.map.addLayer(dem, {'min': 0, 'max': 1000}, 'elevation [meters]')
map_panel_2.map.addLayer(slope, {'min': 0, 'max': 30}, 'slope [degrees]')

Try out the inspector, by clicking on a point on the map.

## Image Math

Here we will calculate the aspect of our DEM. The aspect is the orientation of the slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.

For fun, and to demonstrate chaining methods together, we will also compute the sin of the aspect. The sin of the aspect (when using radians) represents the "eastness", with +1 being directly east and -1 being directly west.

In [ ]:
# Get the aspect (in degrees).
aspect = ee.Terrain.aspect(dem)

# Convert to radians, compute the sin of the aspect.
sinImage = aspect.divide(180).multiply(math.pi).sin()

### Display maps side-by-side 🐍

We can also display multiple maps in a cell output. Here we will display the DEM on the left and the sin of ths aspect of the DEM on the right. Here is how to (roughly) interpret the colors:

| West  | North/South | East |
| ----- | ----------- | ---- |
| green | white       | blue |

We picked Mount Shasta because of it's unique east/west facing mountain faces.

In [ ]:
map_params_mount_shasta = {
    'center': (41.40902, -122.19492),
    'zoom':10
}
map2b = Map(**map_params_mount_shasta)
map2b.addLayer(dem, {'min': 0, 'max': 4000}, 'elevation [meters]')
map2c = Map(**map_params_mount_shasta)
map2c.addLayer(
    sinImage,
    {'min': -1, 'max': 1,
     'palette':['green', 'white', 'blue']},
    'sine of aspect'
)
widgets.HBox([map2b, map2c])

The map objects have properties that can be queried (or set) from Python code. For example, the following code prints out the center coordinates (lon, lat) of the map.

In [ ]:
map2b.center

The properties of maps can be linked together, in order to sychronise the behavior. Run the following cell, then zoom and/or pan one of the maps.

In [ ]:
def syncronize_maps(map_1, map_2):
  map_center_link = widgets.link((map_1, 'center'), (map_2, 'center'))
  map_zoom_link = widgets.link((map_1, 'zoom'), (map_2, 'zoom'))

syncronize_maps(map2b, map2c)

## Image statistics

We will explore image statistics by:

1.   creating a map
2.   creating a custom geometry by adding points on the map
3.   calculating the stats of that custom geometry by calling `reduceRegion` on it.

We will ultimately answer the question, "what is the mean elevation?" in the custom geometry that we defined.



### Draw Control: Create a Custom Geometry on a Map

In this section, we will create a custom geometry on the map using `ipyleflet.DrawControl` tool. If a geometry wasn't created using the tool, a default geometry is set.

In [ ]:
# Use ipyleaflet to add the ability to draw a geometry on our map.
draw_control = ipyleaflet.DrawControl(
    rectangle={},
    polyline={},
    circlemarker={},
)
def handle_draw(target, action, geo_json):
    with output:
      output.clear_output()
      pprint(geo_json)
draw_control.on_draw(handle_draw)

map2d = Map(**map_init_params)
output = widgets.Output(layout={'border': '1px solid black', 'width':"200"})
map2d.addLayer(dem, {'min': 0, 'max': 1000}, 'elevation [meters]')
map2d.add_control(draw_control)
widgets.VBox([map2d, output])

The last geometry drawn on the map can be queries as follows:

In [ ]:
geom_clientside = draw_control.last_draw['geometry']
geom_clientside

It is possible that the preceding cell was run before any geometry was drawn on the map, so lets specify a default geometry in case that happens.

In [ ]:
# If no geometry was drawn on a map, use a default geometry.
if not geom_clientside:
  geom_clientside = {'type': 'Polygon',
    'coordinates': [[[-112.89, 36.46],
                     [-113.08, 36.18],
                     [-112.67, 36.16],
                     [-112.89, 36.46]]]}

  geom_clientside

In [ ]:
# Create an Earth Engine server-side geometry
geom = ee.Geometry(geom_clientside)

### Spatial reduction: Calculate Stats for Our Custom Geometry

We will use `reduceRegion` to calculate the mean elevation (in meters) for our custom geometry (that was created in the last section).

In [ ]:
# Compute the mean elevation in the polygon.
meanDict = dem.reduceRegion(
  reducer=ee.Reducer.mean(),
  geometry=geom,
  scale=90,
  bestEffort=True
)

# Get the mean from the dictionary and print it.
mean = meanDict.get('elevation')
print('Mean elevation', mean.getInfo())

# Image Collections

In this section we are going to use image collections to find the most cloudy and least cloudy images of Mountain View according to [Landsat 8](https://www.usgs.gov/landsat-missions/landsat-8-data-users-handbook) satelite imagery from 2016.

Specifically, we will use the [Landsat 8 Collection 2 Top of Atmosphere (TOA)](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA) image collection. (TOA images have not been atmospherically corrected.)

In [ ]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

In [ ]:
# Create a geometry that is specified by the Geo for Good venue in Mountain View.
point = ee.Geometry.Point(-122.0648754, 37.4225866)

# Define a default visualization parameters for the Landsat image.
landsat_rgb_viz = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
}

# Filter by our Mountain View coordinates and by the year 2016, which was
# arbitrarily chosen.
spatialFiltered = landsat8.filterBounds(point)
temporalFiltered = spatialFiltered.filterDate('2016-01-01', '2016-12-31')

# Sort based on the amount of cloud cover.
least_cloudy_image = temporalFiltered.sort('CLOUD_COVER').first()
most_cloudy_image = temporalFiltered.sort('CLOUD_COVER', opt_ascending=False).first()

map_most_cloudy = Map(**map_init_params)
map_least_cloudy = Map()
map_most_cloudy.addLayer(most_cloudy_image, landsat_rgb_viz)
map_least_cloudy.addLayer(least_cloudy_image, landsat_rgb_viz)
syncronize_maps(map_most_cloudy, map_least_cloudy)

widgets.VBox([
    widgets.Label(f"CLOUD_COVER (most) = {most_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_most_cloudy,
    widgets.Label(f"CLOUD_COVER (least) = {least_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_least_cloudy
],
layout=widgets.Layout(max_height="600px")
)

## Compositing and Mosaicking

Compositing, masking and mosaicking are different technqiues that we use to process image collections.

**Compositing** refers to the process of aggregating individual pixel values in a collection. The median is often used in composites to remove the effects of cloud cover (bright pixels) and shadows (dark pixels).

In **mosaics**, individual images are stitched together (side by side). Often it is the images that were most recent that are stitched together. We will be using the Landsat 8 dataset. You can understand why the mosaic looks the way it does by taking a look at the [Landsat orbit](https://www.youtube.com/watch?v=yPF2jpjB3Qw).

We will first calculate the composite and the mosaic for Lansat 8 data for the year 2016 (year is arbitrarily chosen) for the point centered around the Geo for Good venue in Mountain View:

In [ ]:
# Filter by the year 2016 (arbitrarily chosen).
temporalFiltered = landsat8.filterDate('2016-01-01', '2016-12-31')
# Calculate the mosaic.
mosaic = temporalFiltered.mosaic()
# Calculate the composite by getting the median over time, for each band, in
# each pixel.
median = temporalFiltered.median()

We will display the mosaic on the right and the composite on the left:

In [ ]:
# Compare the mosaic and composite results.
map4a = Map(**map_init_params)
map4a.addLayer(mosaic, landsat_rgb_viz, 'Landsat 8 (mosaic)')
map4b = Map(**map_init_params)
map4b.addLayer(median, landsat_rgb_viz, 'Landsat 8 (median)')
syncronize_maps(map4a, map4b)
widgets.HBox([map4a, map4b])

Why does the mosaic have white lines?

In [ ]:
HTML(
  '<iframe width="640" height="385" '
  'src="https://www.youtube.com/embed/yPF2jpjB3Qw" '
  'frameborder="0"></iframe>')

## Masking

Masking pixels in an image makes those pixels transparent and excludes them from analysis. Pixels with a mask values of 0 or below will be transparent, mask values between 0 and 1 will be partially rendered, whereas mask values above 1 will be fully rendered.

In this example we will use a mask to only look at land data (exclude the water data) of the [Hansen Global Forest Change](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2021_v1_9) dataset. This dataset is used because in the `datamask` column, water has a value of 2, land has the value 1, and 'no data' has the value 0.

In [ ]:
# Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')

# Select the land/water mask.
datamask = hansenImage.select('datamask')

# Create a binary mask. This means we are only selecting the land pixels (based
# on how the datamask column is defined in the dataset).
mask = datamask.eq(1)

# Update the composite mask with the water mask.
maskedComposite = median.updateMask(mask)

map4c = Map(**map_init_params)
map4c.addLayer(maskedComposite, landsat_rgb_viz, 'masked')
map4c

# NDVI, Mapping a Function over a Collection, Quality Mosaicking

In this section we will calculate the Normalized Difference Vegetation Index (NDVI) for Landsat 8 images.

The NDVI is used to determine how much green vegetation exists in an area. NDVI relies on green vegetation having a strong reflectance for Near Infrared (NIR) and a weak reflectance for red light.





The formula for NDVI is as follows:

\begin{align}
\text{NDVI} = \frac{\text{NIR}-\text{R}}{\text{NIR}+\text{R}}
\end{align}

where $\text{NIR}$ and $\text{R}$ are the spectral reflectance in the near-infrared and red (visible) regions, respectively (source: [wikipedia](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index)).

For Landsat 8 and 9:

\begin{align}
\text{NDVI} = \frac{\text{Band5}-\text{Band4}}{\text{Band5}+\text{Band4}}
\end{align}

where $\text{Band5}$ and $\text{Band4}$ are the corresponding Landsat bands, respectively (source: [USGS](https://www.usgs.gov/landsat-missions/landsat-normalized-difference-vegetation-index#:~:text=In%20Landsat%204%2D7%2C%20NDVI,Band%205%20%2B%20Band%204)).


Below are examples of NDVI calculated for two pieces of vegetation in different states.

![NDVI](https://earthobservatory.nasa.gov/ContentFeature/MeasuringVegetation/Images/ndvi_example.jpg)

*Image source: [earthobservatory.nasa.gov](https://earthobservatory.nasa.gov/ContentFeature/MeasuringVegetation/Images/ndvi_example.jpg)*

### Calculate NDVI on a Single Image

First we will calculate the NDVI on a single image:

In [ ]:
# Define a point of interest.
point = ee.Geometry.Point(location_lonlat)

# Get the least cloudy image in 2016.
image = ee.Image(
  landsat8.filterBounds(point)
          .filterDate('2016-01-01', '2016-12-31')
          .sort('CLOUD_COVER')
          .first()
)

NDVI can be calculated within Earth Engine as follows:

In [ ]:
nir = image.select('B5')
red = image.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

Let's display a map showing the `ndvi` object. We will use simple visualization  where blue is low (negative) NDVI and green is high (positive) NDVI. Water tends to result in a negative NDVI value, while clouds have NDVI values near zero.

In [ ]:
ndvi_vis_arams = {
    'bands': 'NDVI',
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

map5a = Map(**map_init_params)
map5a.addLayer(ndvi, ndvi_vis_arams, 'NDVI image')
map5a

Because calculating band ratios (such as NDVI) is commonly done as part of a remote sensing analysis workflow, Earth Engine images have a shortcut method to make this easier: `ee.Image.normalizedDifference()`.

In [ ]:
# Remove the NDVI layer
map5a.remove_layer(map5a.layers[1])

In [ ]:
# Redefine NDVI using ee.Image.normalizedDifference, and add it back again.
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
map5a.addLayer(ndvi, ndvi_vis_arams, 'NDVI image')

## Applying NDVI to an Image Collection

In Earth Engine we can apply an algorithm that works on a single image (such as calculating NDVI) to all images in a collection. To do this, we first define a Python function that that operates on a single image. In this specfic example, the function takes an image, calculates NDVI, appends it to the image, and then returns the new image.

In [ ]:
def add_ndvi(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

To test it out, we can apply the `add_ndvi` function to a single image. *(This is a very useful pattern for testing and debugging functions that you write)*.

In [ ]:
ndvi = add_ndvi(image)

We can add this NDVI image to a map/inspector panel, and then use the inspector to confirm that the NDVI band was added.

In [ ]:
map_panel_5 = MapWithInspector(**map_init_params)
map_panel_5.map.addLayer(ndvi, ndvi_vis_arams, 'NDVI')
map_panel_5

But it is even more useful to "map" (i.e. apply) the function over all the images in an image collection:

In [ ]:
with_ndvi = landsat8.map(add_ndvi)

We can then mosaic the images together, and display it. Note that many images processed by the `add_ndvi` function are visible.

In [ ]:
map5b = Map(**map_init_params)
map5b.addLayer(with_ndvi.mosaic(), ndvi_vis_arams, 'NDVI mosaic')
map5b.zoom = 8  # zoom out a bit from the default
map5b

Note that the result is pretty noisy, because `.mosaic()` preferentialy selects the latest pixels in the collection (which often may have clouds). We will improve upon this in the next section.

## Make a greenest pixel composite

In this section we will use [`qualityMosaic`](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-qualitymosaic) to get less noisy NDVI data. `qualityMosaic` works by taking the maximum value composite for the band you provide. In our example, this means choosing the pixel with the largest NDVI value. The maximum is taken to avoid areas with clouds, which have very low NDVI.

In [ ]:
greenest = with_ndvi.qualityMosaic('NDVI')

In [ ]:
map5c = Map(**map_init_params)
map5c.addLayer(greenest, landsat_rgb_viz, 'greenest pixel mosaic')
map5c.zoom = 8  # zoom out a bit from the default
map5c

# Exporting Charts and Images





## Charting 🐍

Python has many plotting options, for example: matplotlib, bokeh, altair, plotly, etc. You can refer to [The Python Visualization Landscape](https://www.youtube.com/watch?v=FytuB8nFHPQ) talk from PyCon 2017 for more information on the plotting libraries that Python offers.

The goal of this section is to create a timeseries plot using Altair to showcase one Python plotting library.

We will create a timeseries plot of NDVI for the Geo for Good venue in Mountain View.

In [ ]:
stat_region = ee.Geometry.Point(location_lonlat)
stat_region.getInfo()

Filter the NDVI image collection to only get images that occur in the Geo for Good venue in Mountain View:

In [ ]:
filtered = with_ndvi.filterBounds(stat_region)

Create a function that takes the image and creates an `ee.Feature` with the mean and the image timestamp (to help enable timeseries plotting):

In [ ]:
def reduce_region_function(img):
  """Return a feature containing the mean value of a region and a timestamp."""

  stat = img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=stat_region,
      scale=30
  )
  return ee.Feature(stat_region, stat).set({'millis': img.date().millis()})

The next two code blocks are helper functions that are used to get the feature properties into a dictionary and then the dictionary values into a pandas dataframe:

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_dataframe(fc):
  """Converts a feature collection to a Pandas dataframe."""
  return pd.DataFrame(fc_to_dict(fc).getInfo())

:Here we apply the helper functions defined above to create a Pandas dataframe:

In [ ]:
stat_fc = (
  ee.FeatureCollection(
    filtered.map(reduce_region_function)
  ).filter(
    ee.Filter.notNull(filtered.first().bandNames())
  )
)
df = fc_to_dataframe(stat_fc)
df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')
#df.head()

Next we narrow our scope of the feature collection to only focus on the bands that matter for NDVI, the `B5` (near-infrared), `B4` (red) and `NDVI` bands:

In [ ]:
keys = ['B5', 'B4', 'NDVI']
source = pd.melt(
    df[['B5', 'B4', 'NDVI', 'timestamp']],
    id_vars='timestamp',
    value_vars=keys,
    var_name='band'
)
source.head()  # display the first few values

Here we use the dataframe to plot the timeseries for B5, B4, and NDVI:

In [ ]:
alt.Chart(source).mark_line().encode(
    x='timestamp:T',
    y='value',
    color=alt.Color(
        'band',
        scale=alt.Scale(
            domain=['B4', 'B5', 'NDVI'],
            range=['red', 'purple', 'green']
        )
    ),
    tooltip=['band', 'value', 'timestamp:T']
).interactive(bind_y=False)

## Exporting Images 🐍

This section will demonstrat how to export an image, using the Python client library.

To start, create a 3-band, 8-bit, color-IR composite that we will export.

In [ ]:
visualization = greenest.visualize(**landsat_rgb_viz)

We can preview the visualization by adding it to an interative map.

In [ ]:
map_init_params_eastern_sierra = {
    'center': (37.05, -118.40),
    'zoom':8
}
map6a = Map(**map_init_params_eastern_sierra)
map6a.addLayer(visualization, {}, 'visualization test')
map6a

Next we can create a *task* that can be used to export the image. Note that a task will not run until it is *started*.

In [ ]:
asset_id = 'projects/ee-ee-test-te/assets/temp/g4g22_python_export_example'

task = ee.batch.Export.image.toAsset(
      visualization,
      description='Greenest_pixel_composite',
      assetId=asset_id,
      scale=30,
      region="""[[[-119.2, 37.9],
                  [-119.2, 36.2],
                  [-117.6, 36.2],
                  [-117.6, 37.9]]]"""  # California, East Side of Sierras
  )

The task can take ~10 minutes to finish. For this training, we already ran the task, so the `start()` call has been commented out. However, if you have updated the analysis and want to export a new image, you can uncomment the following line and run it (after first updating the `asset_id` in the previous cell).

In [ ]:
# task.start()

You can check on the status of your task by visitng the Earth Engine task manager webpage: https://code.earthengine.google.com/tasks

In [ ]:
exported_asset = ee.Image(asset_id)

map6b = Map(**map_init_params_eastern_sierra)
map6b.addLayer(exported_asset, {}, 'exported asset')
map6b